# [Репозиторий](https://github.com/Lynxtail/Additional-chapters-of-fundamental-training)

## Задание G, вариант 1

Выполнить хэширование с помощью $T$-функции $H : \{0, 1\}^m \times \{0, 1\}^m \rightarrow \{0, 1\}^m$. Для построения функции $H$ нужно использовать корректно определенные операции (арифметические, поразрядные логические, экспоненцирование, деление).

In [131]:
import bitstring as bs
import matplotlib as plt
import random

### Определение операций

In [132]:
# наложение маски a and C
def mask_(a:bs.BitArray, C:bs.BitArray='1111') -> bs.BitArray:
    return a & C

In [146]:
# экспоненцирование x |^ y := (1 + 2x)^y
def expo_(x:int, y:int) -> int:
    return (1 + 2 * x)**y

In [147]:
# деление x/y := x(y |^ (-1)) = x / (1 + 2y)
def divide_(x:int, y:int) -> int:
    return x * expo_(y, -1)

In [135]:
# редуцирование a mod 2^n
def reduce_(a:bs.BitArray, n:int) -> bs.BitArray:
    return a % 2**n

Построение $T-$ функции $H:\{0, 1\}^m \times \{0,1\}^m \rightarrow \{0,1\}^m$

Например: $$\left( (7 + x^5)^{x^3 \,\text{OR}\,x^7} + 8 \cdot ((x \,\text{XOR}\, 5) - x^3 \cdot (x \,\text{AND}\, x^2)) ^ \frac{6x^4}{1+2x} \right)^\frac{3x^2 + 8x^5}{9 + 10x^9}$$

In [149]:
# H - T-функция на {0, 1}^m, где m - длина
def H_1(x:bs.BitArray) -> bs.BitArray:
    # e.g.
    return expo_(expo_(7 + expo_(x.i, 5), expo_(x.i, 3) | expo_(x.i, 7)) + 
        8 * ((x ^ 5) - expo_(x.i, 3) * expo_(x & expo_(x.i, 2), divide_(6 * expo_(x.i, 4), 1 + 2 * x))), 
            divide_(3 * expo_(x.i, 2) + 8 * expo_(x.i, 5), 9 + 10 * expo_(x.i, 9)))

In [150]:
# H - T-функция на {0, 1}^m, где m - длина
def H_2(x:bs.BitArray, y:bs.BitArray) -> bs.BitArray:
    # e.g.
    return (x ^ y) + (1 + expo_(x.i, 2)) 

Построение графика

In [138]:
# построение графика T-функции
def draw(x:list[int], y:list[int]) -> None:
    plt.scatter(x, y, s=1)
    plt.title(f'')
    plt.show()

Ввод исходного сообщения:

In [139]:
# w - исходное сообщение
# ввести между кавычек
w = 'asfgagas sgasdgsagdz'


In [140]:
# h_0 - произвольный вектор длины m
m = 3
h_0 = [random.randint(0, 1) for _ in range(m)]
h_0 = ''.join(map(str, h_0))
h_0 = bs.BitArray(bin=h_0)


In [141]:
# разбиение сообщения w на блоки a_i длины m каждый
# w = [a_k]...[a_2][a_1]
a = '.'.join(map(bin, bytearray(w, 'utf8')))[2:].replace('.0b', '')
a = [a[i : i + m] for i in range(0, len(a), m)]
# добавление незначащих нулей к крайнему a
if len(a[-1]) < m:
    a[-1] = ''.join(['0']*(m - len(a[-1]))) + a[-1]
for i in range(len(a)):
    a[i] = bs.BitArray(bin=a[i])

In [152]:
# h_{i+1} = H(a_{i+1}, h_i), i = 0, 1, ...
# h_k - хеш-значение сообщения
k = len(a)
h = [h_0]
for i in range(k):
    print(f'h{i + 1} = H({a[i], h[-1]})')
    h.append(H_2(a[i], h[-1]))
h

h1 = H((BitArray('0b110'), BitArray('0b001')))
h2 = H((BitArray('0b000'), BitArray('0b1110000000000')))


ValueError: Bitstrings must have the same length for ^ operator.